# API Testing Notebook
Test all API clients and validate data retrieval

In [ ]:
import sys
sys.path.append('..')

import os
from dotenv import load_dotenv
import pandas as pd
from loguru import logger

# Load environment variables
load_dotenv()

# Configure logger
logger.add("../logs/api_testing.log", rotation="10 MB")

print("Environment loaded successfully")

## Test 1: FCC Client

In [ ]:
from src.api_clients import FCCClient

# Initialize FCC client
fcc = FCCClient()

# Create sample database
print("Creating sample FCC database...")
sample_df = fcc.create_sample_database()
print(f"\nSample database created with {len(sample_df)} towers")
print("\nSample data:")
print(sample_df)

In [ ]:
# Test: Query towers in radius
test_lat, test_lon = 37.7749, -122.4194  # San Francisco
radius_km = 5

print(f"\nQuerying towers within {radius_km}km of ({test_lat}, {test_lon})...")
towers = fcc.get_towers_in_radius(test_lat, test_lon, radius_km)

print(f"\nFound {len(towers)} towers:")
print(towers[['registration_number', 'owner_name', 'height_feet', 'distance_km']])

In [ ]:
# Test: Tower density calculation
density = fcc.get_tower_density(test_lat, test_lon, radius_km)
print(f"\nTower density: {density:.4f} towers per sq km")

## Test 2: OpenCelliD Client

In [ ]:
from src.api_clients import OpenCelliDClient

# Get API key from environment
opencellid_key = os.getenv('OPENCELLID_API_KEY')

if not opencellid_key:
    print("WARNING: OPENCELLID_API_KEY not set in .env file")
    print("Get your free API key from: https://opencellid.org/register")
else:
    # Initialize client
    opencellid = OpenCelliDClient(api_key=opencellid_key)
    print("OpenCelliD client initialized")

In [ ]:
# Test: Count towers in radius
if opencellid_key:
    print(f"\nQuerying OpenCelliD towers within 3km of ({test_lat}, {test_lon})...")
    count = opencellid.count_towers_in_radius(test_lat, test_lon, radius_km=3)
    print(f"Found {count} cell towers")
    
    # Get enrichment features
    features = opencellid.get_enrichment_features(test_lat, test_lon, radius_km=3)
    print("\nOpenCelliD Features:")
    for key, value in features.items():
        print(f"  {key}: {value}")

## Test 3: Census API Client

In [ ]:
from src.api_clients import CensusClient

# Get API key from environment
census_key = os.getenv('CENSUS_API_KEY')

if not census_key:
    print("WARNING: CENSUS_API_KEY not set in .env file")
    print("Get your free API key from: https://api.census.gov/data/key_signup.html")
else:
    # Initialize client
    census = CensusClient(api_key=census_key)
    print("Census client initialized")

In [ ]:
# Test: Get demographics for location
if census_key:
    print(f"\nGetting demographics for ({test_lat}, {test_lon})...")
    demographics = census.get_demographics_by_location(test_lat, test_lon)
    
    print("\nDemographic Data:")
    for key, value in demographics.items():
        if isinstance(value, float):
            print(f"  {key}: {value:,.0f}")
        else:
            print(f"  {key}: {value}")

In [ ]:
# Test: Get enrichment features
if census_key:
    features = census.get_enrichment_features(test_lat, test_lon, radius_km=3)
    
    print("\nCensus Enrichment Features:")
    for key, value in features.items():
        if isinstance(value, float):
            print(f"  {key}: {value:,.0f}")
        else:
            print(f"  {key}: {value}")

## Test 4: Combined Enrichment

In [ ]:
# Combine all features for a single location
def enrich_location(lat, lon, radius_km=3):
    """Get all enrichment features for a location"""
    features = {
        'latitude': lat,
        'longitude': lon,
        'radius_km': radius_km
    }
    
    # FCC features
    fcc_count = fcc.count_towers_nearby(lat, lon, radius_km)
    fcc_density = fcc.get_tower_density(lat, lon, radius_km)
    features['fcc_tower_count'] = fcc_count
    features['fcc_tower_density'] = fcc_density
    
    # OpenCelliD features (if API key available)
    if opencellid_key:
        opencellid_features = opencellid.get_enrichment_features(lat, lon, radius_km)
        features.update(opencellid_features)
    
    # Census features (if API key available)
    if census_key:
        census_features = census.get_enrichment_features(lat, lon, radius_km)
        features.update(census_features)
    
    return features

# Test with sample location
print("\n" + "="*60)
print("COMBINED ENRICHMENT TEST")
print("="*60)

enriched = enrich_location(test_lat, test_lon, radius_km=3)

print(f"\nEnriched features for ({test_lat}, {test_lon}):")
for key, value in enriched.items():
    if isinstance(value, float):
        print(f"  {key}: {value:,.4f}")
    elif isinstance(value, int):
        print(f"  {key}: {value:,}")
    else:
        print(f"  {key}: {value}")

## Test 5: Multiple Locations

In [ ]:
# Test with multiple locations
test_locations = [
    {'name': 'San Francisco', 'lat': 37.7749, 'lon': -122.4194},
    {'name': 'Los Angeles', 'lat': 34.0522, 'lon': -118.2437},
    {'name': 'New York', 'lat': 40.7128, 'lon': -74.0060},
]

results = []

for loc in test_locations:
    print(f"\nEnriching {loc['name']}...")
    features = enrich_location(loc['lat'], loc['lon'], radius_km=3)
    features['city'] = loc['name']
    results.append(features)

# Convert to DataFrame
df_results = pd.DataFrame(results)

print("\n" + "="*60)
print("ENRICHMENT RESULTS")
print("="*60)
print(df_results.T)

## Summary

This notebook tested:
1. ✅ FCC Client - Tower data from local database
2. ✅ OpenCelliD Client - Cell tower locations (requires API key)
3. ✅ Census Client - Demographics (requires API key)
4. ✅ Combined enrichment pipeline
5. ✅ Batch processing multiple locations

Next steps:
- Analyze data structure and distributions
- Create cross-sectional dataset
- Build feature engineering pipeline